In [1]:
!pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 39.1 MB/s eta 0:00:00


In [2]:
import numpy as numpy
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
pre_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoded_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [4]:
bert_pre_model=hub.KerasLayer(pre_url)

In [5]:
test_text=["nice movie indeed","I love python"]
text_pre=bert_pre_model(test_text)
text_pre.keys()

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])

In [10]:
text_pre['input_word_ids'],text_pre["input_mask"],text_pre["input_type_ids"]

(<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
 array([[  101,  3835,  3185,  5262,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,

In [12]:
bert_model=hub.KerasLayer(encoded_url)

In [13]:
model_result=bert_model(text_pre)
model_result.keys()

dict_keys(['encoder_outputs', 'default', 'sequence_output', 'pooled_output'])

In [20]:
model_result["default"],model_result["pooled_output"],model_result["encoder_outputs"],model_result["sequence_output"].shape

(<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
 array([[-0.79177403, -0.21411912,  0.49769545, ...,  0.24465235,
         -0.47334474,  0.81758696],
        [-0.86115324, -0.23277396,  0.38201126, ...,  0.4286716 ,
         -0.6130618 ,  0.86048734]], dtype=float32)>,
 <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
 array([[-0.79177403, -0.21411912,  0.49769545, ...,  0.24465235,
         -0.47334474,  0.81758696],
        [-0.86115324, -0.23277396,  0.38201126, ...,  0.4286716 ,
         -0.6130618 ,  0.86048734]], dtype=float32)>,
 [<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
  array([[[ 0.1290144 ,  0.00644745, -0.03614967, ...,  0.04999624,
            0.06149195, -0.02657542],
          [ 1.1753386 ,  1.2140785 ,  1.1569982 , ...,  0.11634357,
           -0.3585537 , -0.40490186],
          [ 0.03859041,  0.5386997 , -0.2108978 , ...,  0.21858197,
            0.7260168 , -1.1158606 ],
          ...,
          [-0.07587015, -0.25421894,  0.7075512 , ...,  0.505419








now real world problem

In [21]:
import pandas as pd

In [22]:
df=pd.read_csv("/content/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
df.groupby("Category").describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [24]:
df_ham=df[df["Category"]=="ham"]
df_ham.shape

(4825, 2)

In [25]:
df_spam=df[df["Category"]=="spam"]
df_spam.shape

(747, 2)

In [26]:
df_ham=df_ham.sample(df_spam.shape[0])
df_ham.shape

(747, 2)

In [27]:
x=pd.concat([df_spam,df_ham])
x = x.sample(frac=1).reset_index(drop=True)
x

,Category,Message
0,spam,"Thanks for your ringtone order, reference numb..."
1,spam,WIN a £200 Shopping spree every WEEK Starting ...
2,spam,URGENT! This is the 2nd attempt to contact U!U...
3,ham,"Hey babe, my friend had to cancel, still up fo..."
4,spam,Please call our customer service representativ...
...,...,...
1489,ham,I not busy juz dun wan 2 go so early.. Hee..
1490,ham,I need details about that online job.
1491,ham,If I die I want u to have all my stuffs.
1492,ham,"Love isn't a decision, it's a feeling. If we c..."


In [28]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x.Category=le.fit_transform(x.Category)
x=x.rename(columns={"Category":"spam"})

In [29]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x.Message,x.spam,stratify=x["spam"])

In [30]:
def get(sen):
  pre=bert_pre_model(sen)
  return bert_model(pre)["pooled_output"]

get([
      "i will die",
      "Discount unlocked"
])


<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.85732603, -0.23019192,  0.42889562, ...,  0.44638506,
        -0.61736983,  0.88845825],
       [-0.90350866, -0.53026074, -0.8121572 , ..., -0.44483015,
        -0.7480224 ,  0.9483615 ]], dtype=float32)>

In [31]:
e=get([
    "bANNANA",
    "apple",
    "cricket",
    "dhoni"
])
e

<tf.Tensor: shape=(4, 768), dtype=float32, numpy=
array([[-0.8119131 , -0.22548981,  0.25914973, ...,  0.2680573 ,
        -0.55815613,  0.77699494],
       [-0.8196456 , -0.29609606,  0.20951797, ...,  0.25593394,
        -0.5874299 ,  0.8434556 ],
       [-0.92026347, -0.40818018, -0.26689315, ..., -0.19524567,
        -0.62910986,  0.8874004 ],
       [-0.8921565 , -0.5717125 , -0.7705461 , ..., -0.40886942,
        -0.73605114,  0.88198   ]], dtype=float32)>

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[1],e[3]])[0][1]

0.8805474

In [42]:
# Bert layers
text_input=tf.keras.layers.Input(shape=(),dtype=int,name="text")
preprocessed_text = bert_pre_model(text_input)
en=bert_model(preprocessed_text)
#NN layers
l=tf.keras.layers.Dropout(0.1,name="dropout")(en["pooled_output"])
l=tf.keras.layers.Dense(1,activation="sigmoid",name="output")(l)
#model
model=tf.keras.Model(inputs=[text_input],outputs=[l])

ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=int64, sparse=None, name=text>
  • training=None

In [68]:
model.summary()

NameError: name 'model' is not defined

In [45]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

# Wrap bert_pre_model in a Lambda layer to handle preprocessing within the model
# Define the output shape of the bert_pre_model
preprocessed_text = tf.keras.layers.Lambda(lambda x: bert_pre_model(x), output_shape=(768,))(text_input) # Assuming your bert_pre_model outputs a tensor with shape (768,)

en = bert_model(preprocessed_text)

# NN layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(en["pooled_output"])
l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

# model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

ValueError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 768), dtype=float32, sparse=False, name=keras_tensor_3>
  • training=None